### **IMPORTS**

In [1]:
# %pip install igraph
# %pip install graphviz

In [2]:
import numpy as np
import re
import json
import igraph as ig
from graphviz import Digraph

### **Helper Functions**

### **Check Regex Validity**

In [3]:

def is_regex_valid(regex):

	# 1- Check that the characters in regex are within the valid set of characters
	# 2- check that all brackets are closed
	regex_operations = ['|', '(', ')', '[', ']', '.', '?', '*', '+', '-', '\\\\']
	bracket,parenthesis = 0, 0

	for char in regex:
		if not char.isalnum() and char != ' ' and char not in regex_operations:
			return False
		
		if char == '(':
			bracket += 1
		elif char == ')':
			bracket -= 1
		elif char == '[':
			parenthesis += 1
		elif char == ']':
			parenthesis -= 1
	if bracket != 0 or parenthesis != 0:
		return False
	
	return True

def is_regex_validdd(regex):
    try:
        re.compile(regex)
        return True
    except:
        return False
# Test
validity_check = is_regex_valid("[A-Zl]/ko;]")
print(validity_check)

False


### **Change regex to PostFix (Shunt_Yard)**

In [4]:
def regex_to_postfix(regex):
	# Operators and precedance level: * (kleene star), + (one or more), ? (zero or one), . (concatenation), and | (ORing).
	operators = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
	# Initialize the postfix and stack (temp) strings to empty strings.
	postfix, stack = "", ""
	# 1. Check for square brakcets of letter as ORs and replace them with |. i.e. Character class
	for i in range(len(regex)):
		char = regex[i]
		if char == '[':
			j = i + 1
			while regex[j] != ']':
				if regex[j].isalnum() and regex[j + 1].isalnum():
					regex = regex[:j + 1] + '|' + regex[j + 1:]
				j += 1

	# Replace all remaining square brackets with parentheses.
	# This is done because parentheses are used to group sub-expressions in regular expressions
	regex = regex.replace('[', '(')
	regex = regex.replace(']', ')')

	# print("postfix: ", regex)

	# Replace ranges in character classes with the individual characters they represent.
	hyphen_count = regex.count('-')
	for i in range(hyphen_count):
		for j in range(len(regex)):
			char = regex[j]
			if char == '-':
				final = regex[j + 1]
				first = regex[j - 1]
				temp_list = ''
				for k in range(int(ord(final) - ord(first))):
					temp_list = temp_list + '|'
					temp = chr(ord(first) + k + 1)
					temp_list = temp_list + temp
				regex = regex[0: j] + temp_list + regex[j + 2:]
				break
	# print("postfix2", regex)
	
	# Insert a concatenation operator (.) between any two adjacent characters if there is no operator between them. OR there is a bracket
	dotIndices = []
	for i in range(len(regex) - 1):
		startOps = [')', "*","+", "*"]
		endOps = ["*", "+", ".", "|", ")"]
		if regex[i] in startOps and regex[i+1] not in endOps:
			dotIndices.append(i)
		elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
			dotIndices.append(i)
	
	for i in range(len(dotIndices)):
		regex = regex[:dotIndices[i] + 1 + i] + '.' + regex[dotIndices[i] + 1 + i:]
	# print("postfix: ", regex)
	

	# Shunt_Yard Algorithm
	for i in range(len(regex)):
		c = regex[i]
		# If the character is an opening parenthesis, push it onto the stack.
		if c == '(':
			stack = stack + c
		# If the character is a closing parenthesis, pop operators off the stack and append them to the postfix string until an opening parenthesis is found & delete the parenthesis
		elif c == ')':
			while stack[-1] != '(':
				# places the character at the end of the stack in the postfix expression
				postfix = postfix + stack[-1]
				# [:-1] denotes up to or including the last character
				stack = stack[:-1]
			stack = stack[:-1]  # removes the open bracket in the stack

		# If the character is an operator, pop operators off the stack and append them to the postfix string as long as they have higher or equal precedence to the current operator. Then push the current operator onto the stack.
		elif c in operators:
			while stack and operators.get(c, 0) <= operators.get(stack[-1], 0):
				postfix, stack = postfix + stack[-1], stack[:-1]
			stack = stack + c

		# If the character is a operand (i.e. not an operator or parenthesis), append it to the postfix string.
		else:
			postfix = postfix + c
	# After iterating over all characters of the regular expression, the function pops any remaining operators off the stack and appends them to the postfix string.
	while stack:
		postfix, stack = postfix + stack[-1], stack[:-1]
	# print("postfix: ", regex)

	# Finally, the function returns the postfix notation of the input regular expression.
	return postfix

# print(regex_to_postfix("[a-c]"))

### **Make a Class for NFA**

In [5]:
# Each state has a list of transitions and epsilon transitions
# We have 2 types of states accepting and non accepting
class State:
	# id = 0 # Smarter than using index in NFA
	def __init__(self, name, start=False, accepting=True, transitions=[], parents=[]):
		# self.id = State.id
		# State.id += 1
		self.name = name
		self.transitions = []
		self.parents = []
		self.accepting = accepting
		self.start = start
	
	def add_transition(self, transition, state):
		self.transitions.append((transition,state))
		state.parents.append(self)
		self.accepting = False

	def get_transitions(self):
		return self.transitions.copy()
	
	def get_parents(self):
		return self.parents.copy()
	
	# Print states override
	# def __str__(self):
	# 	output = "State: " + str(id(self)) + "\n"
	# 	output += "  Transitions: " + str(self.transitions) + "\n"
	# 	output += "  Epsilon transitions: " + str(self.epsilon_transitions) + "\n"
	# 	output += ("  Accepting? " + str(self.accepting) + "\n")
	# 	output += ("  Saart? " + str(self.accepting) + "\n")
	# 	return output

In [6]:
# NFA class
	# Consists of
		# 1. States ( eaach has its transitions and epsilon transitions)
		# 2. Start State
		# 3. Final State			 

	# Operations that can be done on them
		# 1. Concatenation
		# 2. Union
		# 3. Kleene Star
		# 4. Positive Closure

class NFA:
	def __init__(self, start_state=None, accept_state=None, regex=None):
		self.start_state = start_state
		self.accept_state = accept_state
		if not start_state and not accept_state and regex:
			nfa = self.create_nfa(regex)
			self.start_state = nfa.start_state
			self.accept_state = nfa.accept_state

	def create_nfa(self, regex):
		"""
		Converts a regular expression in postfix notation to an NFA.
		"""
		NFAStack, index = [], 0

		for symbol in regex:
			if symbol == '.':  # Concatenation
				# Pop the operands
				nfa2 = NFAStack.pop()
				nfa1 = NFAStack.pop()
				#Concatenate with an epsilon transition
				nfa1.accept_state.add_transition('ϵ', nfa2.start_state)
				# Add to NFA Stack output
				NFAStack.append(NFA(nfa1.start_state, nfa2.accept_state))

			elif symbol == '|':  # Union
				# Pop the operands
				nfa2 = NFAStack.pop()
				nfa1 = NFAStack.pop()
				# we have a start and an end (between which we put the symbols)
				start = State("S"+str(index))
				accepting = State("S"+str(index+1))
				# Add the 2 paths with epsilon transition from start
				start.add_transition('ϵ',nfa1.start_state)
				start.add_transition('ϵ',nfa2.start_state)
				# Add the 2 paths with epsilon transition to the end
				nfa1.accept_state.add_transition('ϵ', accepting)
				nfa2.accept_state.add_transition('ϵ', accepting)
				# Add to NFA Stack output
				NFAStack.append(NFA(start,accepting))
				index+=2

			elif symbol == '*':  # Kleene star
				# Pop the operand
				nfa = NFAStack.pop()
				# we have a start and an end 
				start = State("S"+str(index))
				accepting = State("S"+str(index+1))
				# Add the 2 paths one with symbol and one empty to the end
				start.add_transition('ϵ',nfa1.start_state)
				start.add_transition('ϵ',accepting)
				# Add the 2 paths with epsilon transition to the end and back again to start
				nfa.accept_state.add_transition('ϵ', start)
				nfa.accept_state.add_transition('ϵ', accepting)
				# Add to NFA Stack output
				NFAStack.append(NFA(start,accepting))
				index+=2

			elif symbol == '+':  # Positive closure (A+)
				nfa = NFAStack.pop()  
				# we have a start and an end 
				start = State("S"+str(index))
				accepting = State("S"+str(index+1))
				# Add the path one with symbol 
				start.add_transition('ϵ',nfa.start_state)
				# Add the 2 paths with epsilon transition to the end  and back again to start
				nfa.accept_state.add_transition('ϵ', start)
				nfa.accept_state.add_transition('ϵ', accepting)
				# Add to NFA Stack output
				NFAStack.append(NFA(start,accepting))
				index+=2

			elif symbol == '?':  # Kleene star
				nfa = NFAStack.pop()  
				# we have a start and an end 
				start = State("S"+str(index))
				accepting = State("S"+str(index+1))
				# Add the 2 paths one with symbol and one empty to the end
				start.add_transition('ϵ',nfa1.start_state)
				start.add_transition('ϵ',accepting)
				# Add the path with epsilon transition to the end
				nfa.accept_state.add_transition('ϵ', accepting)
				# Add to NFA Stack output
				NFAStack.append(NFA(start,accepting))
				index+=2
				
			else:  # character/symbol/number
				start = State("S"+str(index))
				accepting = State("S"+str(index+1))
				# Add an epsilon transition
				start.add_transition(symbol,accepting)
				# Add to NFA Stack output
				NFAStack.append(NFA(start,accepting))
				index+=2
		return NFAStack.pop()  # Final NFA
	
	def get_states(self):
		'''
		Return states in the NFA as a list
		'''
		visited, statesList, queue = set(), [], [self.start_state]
		visited.add(self.start_state)
		while queue:
			state = queue.pop(0)
			statesList.append(state)
			for (transition) in state.transitions:
				if transition[1] not in visited:
					visited.add(transition[1])
					queue.append(transition[1])
		states={}
		for state in statesList:
			stateDictionary = {
                'isTerminatingState': state.accepting,
            }
			for symbol, nextState in state.transitions:
				if symbol not in stateDictionary:
					stateDictionary[symbol] = nextState.name
				else:
					stateDictionary[symbol] += ',' + nextState.name
			states[state.name] = stateDictionary
		
		return {'startingState': self.start_state.name,**states,}

	def get_graph(self, name="fsm", view=False):
		'''
		Return the NFA as a graph
		'''
		nfa = self.get_states()
		g = Digraph(engine='dot')
		for state, transitions in nfa.items():
			if state == 'startingState':
				continue
			if transitions['isTerminatingState']:
				g.node(state, shape='doublecircle')
			else:
				g.node(state, shape='circle')
			
			for symbol, nextState in transitions.items():
				if symbol == 'isTerminatingState':
					continue
				childStates = nextState.split(',')
				for child in childStates:
					g.edge(state, child, name=symbol)
		g.render(name, view=view)
		return g
	
	# # Override print
	# def __str__(self):
	# 	return self.get_states()
	




### **Write Result to JSON**

In [7]:
def write_json(nfa, filename = "fsm.json"):
    json_object = json.dumps(nfa, indent = 4) 
    with open(filename, "w") as f:
        json.dump(json_object, f)
    

### **Create Graph**

In [8]:
def display_graph(nfa, filename="fsm.gv"):
    nfa.get_graph(name=filename)
    pass

## **MAIN**

In [9]:
# 1. Get user Input
# regex = input("Enter a regex: ")
regex_1 ='a|b'

# # 2. Check if the regex is valid
if not is_regex_valid(regex_1):
	print("Invalid regex")

# # 3. Turn regex to postfix
postfixRegex = regex_to_postfix(regex_1)
print("postfix regex:", postfixRegex)

# # 4. Turn postfix to NFA
nfa = NFA(regex= postfixRegex)
print("NFA:", nfa.get_states())

# # 5. Write the FSM to a file
write_json(nfa.get_states())

# # 6. Display the NFA as a graph
display_graph(nfa)



NameError: name 'regex' is not defined

In [ ]:
# # 1. Get user Input
# # regex = input("Enter a regex: ")
# regex_1 ='ab(b|c)*d+'

# # # 2. Check if the regex is valid
# if not is_regex_valid(regex):
# 	print("Invalid regex")

# # # 3. Turn regex to postfix
# postfixRegex = regex_to_postfix(regex_1)
# print("postfix regex:", postfixRegex)

# # # 4. Turn postfix to NFA
# nfa = NFA(regex= postfixRegex)
# print("NFA:", nfa.get_states())

# # # 5. Write the FSM to a file
# write_json(nfa.get_states())

# # # 6. Display the NFA as a graph
# display_graph(nfa)

